In [38]:
import numpy as np
import matplotlib.pyplot as plt
from keras.datasets import cifar10
from sklearn.ensemble import RandomForestClassifier

(x_train, y_train), (x_test, y_test) = cifar10.load_data()
x_train = x_train.reshape(x_train.shape[0],32*32*3)
x_test = x_test.reshape(x_test.shape[0],32*32*3)

In [39]:
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train /= 255
x_test /= 255

In [40]:
rf = RandomForestClassifier(max_depth=8)
rf.fit(x_train, y_train)
print("Train :", rf.score(x_train, y_train))
print("Test :", rf.score(x_test, y_test))

/Users/nozawa/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:2: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  


Train : 0.43684
Test : 0.3718


In [ ]:
from sklearn.svm import LinearSVC

x_train_normed = x_train / np.linalg.norm(x_train, ord=2, axis=1, keepdims=True)
x_test_normed = x_test / np.linalg.norm(x_test, ord=2, axis=1, keepdims=True)
svc = LinearSVC()
svc.fit(x_train_normed, y_train)

print("Train :", svc.score(x_train_normed, y_train))
print("Test :", svc.score(x_test_normed, y_test))

/Users/nozawa/anaconda3/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/Users/nozawa/anaconda3/lib/python3.6/site-packages/sklearn/utils/validation.py:578: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [12]:
from keras.models import Sequential
from keras.layers import Dense,Input,Flatten
from keras.utils import np_utils

'''modelの設定'''
model = Sequential()
'''
    modelにレイヤーを追加
    32*32の3チャンネル画像を一次元に伸ばして入力(32*32*3=3072)
'''
model.add(Dense(512, activation='relu',input_shape=(32*32*3,)))
model.add(Dense(256, activation='relu'))
model.add(Dense(128, activation='relu'))
model.add(Dense(10, activation='softmax'))

'''modelに訓練プロセスを設定'''
model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

y_train = np_utils.to_categorical(y_train)
y_test = np_utils.to_categorical(y_test)


''' 訓練データを設定＆学習 '''
model.fit(x_train, y_train, epochs=20, batch_size=32,validation_split=0.2)

''' モデルの評価 '''
loss_and_metrics = model.evaluate(x_test, y_test, batch_size=128)
print(loss_and_metrics)

''' モデルによる予測 '''
classes = model.predict(x_test, batch_size=128)
print(classes)

Train on 40000 samples, validate on 10000 samples
Epoch 1/20
40000/40000 [==============================] - 41s 1ms/step - loss: 1.9002 - acc: 0.3061 - val_loss: 1.8043 - val_acc: 0.3478
Epoch 2/20
40000/40000 [==============================] - 45s 1ms/step - loss: 1.7129 - acc: 0.3814 - val_loss: 1.6907 - val_acc: 0.3880
Epoch 3/20
40000/40000 [==============================] - 44s 1ms/step - loss: 1.6297 - acc: 0.4135 - val_loss: 1.6361 - val_acc: 0.4182
Epoch 4/20
40000/40000 [==============================] - 43s 1ms/step - loss: 1.5582 - acc: 0.4380 - val_loss: 1.5780 - val_acc: 0.4390
Epoch 5/20
40000/40000 [==============================] - 44s 1ms/step - loss: 1.5300 - acc: 0.4508 - val_loss: 1.5934 - val_acc: 0.4245
Epoch 6/20
40000/40000 [==============================] - 41s 1ms/step - loss: 1.4838 - acc: 0.4671 - val_loss: 1.5119 - val_acc: 0.4587
Epoch 7/20
40000/40000 [==============================] - 46s 1ms/step - loss: 1.4511 - acc: 0.4794 - val_loss: 1.5445 - val_acc

In [37]:
from keras.layers import Conv2D, MaxPooling2D

modelConv = Sequential()

modelConv.add(Conv2D(filters=128, kernel_size=(3,3), activation='relu', input_shape=(32, 32, 3)))
modelConv.add(Conv2D(filters=256, kernel_size=(3,3), activation='relu'))
modelConv.add(MaxPooling2D(pool_size=(2,2)))
modelConv.add(Dropout(0.25))
modelConv.add(Flatten())
modelConv.add(Dense(units=128, activation='relu'))
modelConv.add(Dropout(0.5))
modelConv.add(Dense(units=10, activation='softmax'))

modelConv.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

model.fit(x_train, y_train, epochs=10, batch_size=32, validation_split=0.2)

loss_and_metrics = model.evaluate(x_test, y_test, batch_size=128)
print(loss_and_metrics)

classes = model.predict(x_test, batch_size=128)
print(classes)

Train on 40000 samples, validate on 10000 samples
Epoch 1/10
40000/40000 [==============================] - 39s 967us/step - loss: 1.3385 - acc: 0.5169 - val_loss: 1.4934 - val_acc: 0.4744
Epoch 2/10
40000/40000 [==============================] - 39s 979us/step - loss: 1.3201 - acc: 0.5242 - val_loss: 1.5130 - val_acc: 0.4711
Epoch 3/10
40000/40000 [==============================] - 46s 1ms/step - loss: 1.3013 - acc: 0.5324 - val_loss: 1.4943 - val_acc: 0.4757
Epoch 4/10
40000/40000 [==============================] - 36s 889us/step - loss: 1.2845 - acc: 0.5384 - val_loss: 1.5178 - val_acc: 0.4750
Epoch 5/10
40000/40000 [==============================] - 40s 994us/step - loss: 1.2675 - acc: 0.5412 - val_loss: 1.4939 - val_acc: 0.4884
Epoch 6/10
40000/40000 [==============================] - 40s 993us/step - loss: 1.2450 - acc: 0.5487 - val_loss: 1.5286 - val_acc: 0.4826
Epoch 7/10
40000/40000 [==============================] - 40s 1ms/step - loss: 1.2375 - acc: 0.5505 - val_loss: 1.5069